## Prepare

In [ ]:
%%sh

sudo yum install -y yum-utils shadow-utils
sudo yum-config-manager --add-repo https://rpm.releases.hashicorp.com/AmazonLinux/hashicorp.repo
sudo yum -y install terraform

In [ ]:
!terraform version

## policy needed for both samgemaker notebook & terraform
[sagemaker role](sm-exec-role.txt)   
AmazonDynamoDBFullAccess(optional)

## Build & Push Image for Lambda

In [ ]:
%%sh

cd /home/ec2-user/SageMaker/asyncInf

# The name of our algorithm
image_name=sagemaker-kwm-apigw

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

# build & push
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:latest"
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}
docker build -t ${image_name} .
docker tag ${image_name} ${fullname}
docker push ${fullname}


# deploy apigw & lambda
cd /home/ec2-user/SageMaker/asyncInf/infra/
terraform apply -auto-approve 

## Build Infra by Terraform

In [ ]:
%%sh

cd asyncInf/infra/
terraform init

In [ ]:
%%sh

cd asyncInf/infra/
terraform plan

In [ ]:
%%sh

cd /home/ec2-user/SageMaker/asyncInf/infra/
terraform apply -auto-approve 

## Trigger Async Inference

In [ ]:
%%sh

## make sure ecr 'repo name' is the same as 'type' below, in the example, they both are 'sagemaker-kwm-deberta-inf'

## whitelist
#curl -d '{"type":"deberta123"}' -H 'Content-Type: application/json' https://llmm8ax93f.execute-api.us-east-1.amazonaws.com/serverless_lambda_stage/inference

## normal schedule a job, current only one item in whitelist
#curl -d '{"type":"sagemaker-kwm-deberta-inf"}' -H 'Content-Type: application/json' https://llmm8ax93f.execute-api.us-east-1.amazonaws.com/serverless_lambda_stage/inference

curl -d '{"type":"sagemaker-kwm-deberta"}' -H 'Content-Type: application/json' https://xxxxxx.execute-api.us-east-1.amazonaws.com/serverless_lambda_stage/inference    

## you'll get a status 200, then you could go to: https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs for seeing the status of job.

## assum limit is 1, when a job is in processing, if you schedule the job again, will get the information below
## {"status": 502, "errMsg": "Failed to schedule inference at the moment, due to reaching limit 1."}


## TODOs

~~apigw in terraform~~  
~~security - terraform credential~~    
~~req from apigw to lambda~~ 
~~call sagemaker~~  
~~concurrency limit solution (id) - limit should be param  ~~ 
environment vars

status checking  
fsx lustre in container
try code commit integration
blue/green deploy plan
sagemaker project template integration?

tf init deps in china  

In [ ]:
#Deploy:

extract tar.gz, make sure 'ayncInf'

open jupiter and intall terrafrom
setup roles from roles.txt
vars.tf  region & whitelist &role-arn
build & push, region -> cn-north-1

remove terraform states & exec buildAndPush.sh
get enpoint & update cell in jupiter
update region btw in jupiter
